Задание №1

Скачайте текст "Литературных анекдотов". Напишите функцию, которая будет читать файл, лемматизировать текст с помощью pymystem3 и записывать результат в новый файл. У функции должно бы два аргумента: путь к исходному файлу и путь к файлу с лемматизированным текстом. Вызов функции тоже должен быть прописан в решении.

In [109]:
import os, re, json
from pymystem3 import Mystem
from collections import Counter

def lemm_in_file(filecom, fileout): #создаем функцию
    
    with open(filecom, 'r', encoding='utf-8') as f: #открываем и читаем файл
        anecdotes = f.read()
         
    anecdotes_1= [w.strip("!\"#$%&'()*+,./:;<=>?@[\]^_`{|}~„“«»†*—/\-—") for w in anecdotes.split()] #обрезаем пунктуацию
    anecdotes_1 = [w for w in anecdotes_1 if len(w) != 0] #убираем пустоты
    anecdotes_1 = ' '.join(anecdotes_1) #объединяем в строку
    
    m = Mystem() #лемматизируем
    lemmas = m.lemmatize(anecdotes_1)
    anecdotes_lemma = ''.join(lemmas) #совмещаем в строку лемматизированный текст
     
    with open(fileout, 'w', encoding='utf-8') as f: #пишем текст в новый файл
        f.write(anecdotes_lemma)
    
    return anecdotes_lemma
 

In [110]:
lemm_in_file(input("Введите имя начального файла:\n"), input("Введите имя нового файла:\n")) #вызов функции

Введите имя начального файла:
literary_anecdotes.txt
Введите имя нового файла:
loltext.txt


'у вяземский быть квартира окно на тверская бульвар пуш кин очень любить ходить к он в гость прийти и сразу прыг на подоконник свешиваться из окно и смотреть чай он тоже туда на окно подавать иной раз там и заночевать он даже матрац купить специальный только он он не признавать."к что говорить такой роскошь и спихивать матрац с подоконник а потом весь ночь вертеться спать не давать гоголь переодеваться пушкин приходить к пушкин и позвонить пушкин открывать он и кричать смотреть арина родионовна я при идти лермонтов хотеть у пушкин жена уводить на кавказ весь смот рел на она из-за колонна смотреть вдруг устыжаться свой жел ний пушкин думать зеркало русский революция а я свинья пойти вставать перед он на колено и говорить пушкин где твой кинжал вот грудь мой пушкин очень смеяться однажды пушкин стреляться с гоголь пушкин говорить стрелять первый ты как я нет ты ах я нет ты так и не становиться стреляться лев толстой очень любить ребенок однажды он идти по тверской бульвар и увидеть впере

In [111]:
with open ('loltext.txt', 'r', encoding='utf-8') as f:
    anecdotes_lemma = f.read ()
    

In [112]:
anecdotes_lemma = ''.join(anecdotes_lemma) #объединяем в строку

Задание №2

Очистите лемматизированный текст от стоп-слов и посчитайте ipm для оставшихся. Выведите 20 самых частотных по этому параметру слов.

In [113]:
with open('rus_stopwords.txt', 'r', encoding='utf-8') as f: #Открываем и читаем файл со стоп-словами
    stopwords = f.read()
    
    stopwords = stopwords.split('\n') #делаем список по переносу строки

    stopwords_new = [e for e in anecdotes_lemma.split() if e not in stopwords]  #делаем новый список и устанавливаем ограничение на все слова, за исключением стоп-слов                                
    

a_ipm = {} #создаем словарь 
counts = Counter(stopwords_new) #считаем

for k, v in counts.items():
    a_ipm[k] = (v/len(anecdotes_lemma.split()))*1000000
    
sort_ipm = sorted(a_ipm.items(), key=lambda x: x[1], reverse=True) #сортируем
sort_ipm[:20] #выводим 20 самых частотных


[('пушкин', 18987.3417721519),
 ('толстой', 11251.758087201126),
 ('гоголь', 10900.14064697609),
 ('однажды', 10196.905766526019),
 ('лев', 8790.43600562588),
 ('любить', 7032.3488045007025),
 ('достоевский', 6680.731364275668),
 ('тургенев', 5274.261603375528),
 ('ребенок', 4922.644163150492),
 ('царствие', 4571.026722925457),
 ('окно', 4219.4092827004215),
 ('бульвар', 4219.4092827004215),
 ('идти', 4219.4092827004215),
 ('лермонтов', 4219.4092827004215),
 ('приходить', 3867.791842475387),
 ('тверской', 3867.791842475387),
 ('федор', 3867.791842475387),
 ('михайлович', 3867.791842475387),
 ('герцен', 3516.1744022503512),
 ('небесный', 3516.1744022503512)]

Задание №3

Сделайте полный морфологический разбор исходного текста. Напишите регулярное выражение, которое будет извлекать из тега только часть речи. Пройдитесь циклом по списку с разборами, который выдал pymystem3, извлекая из каждого разбора форму слова и его часть речи и записывая их в новый словарь (форма -- ключ, часть речи -- значение). Посчитайте абсолютную частоту для всех частей речи, а затем относительнную частоту (абсолютная частота / длина текста в словах).

In [119]:

m = Mystem()
with open('literary_anecdotes.txt', 'r', encoding='utf-8') as f: #открываем и читаем текст исходного файла
    morf = f.read()
    
morf_analyz = m.analyze(morf)#анализируем
 
vocab = {} #создаём словарь с грамматикой
for word in morf_analyz:
    try:
        form = word['text']
        grammar = word['analysis'][0]['gr']
        vocab[form] = grammar
    except KeyError: #исключаем возможность ошибок
        pass
    except IndexError:
        pass

pos = re.compile('[A-Z]+') #создаем регулярное выражение, извлекающее части речи

newdict_pos = {} #делаем новый пустой словарь

for k, v in vocab.items():#записываем в новый словарь ключ из начального словаря и значение
    newdict_pos[k] = pos.match(v).group(0)
    

In [120]:
newdict_pos


{'У': 'PR',
 'Вяземского': 'S',
 'была': 'V',
 'квартира': 'S',
 'окнами': 'S',
 'на': 'PR',
 'Тверской': 'A',
 'бульвар': 'S',
 'Пуш': 'S',
 'кин': 'S',
 'очень': 'ADV',
 'любил': 'V',
 'ходить': 'V',
 'к': 'PR',
 'нему': 'SPRO',
 'в': 'PR',
 'гости': 'S',
 'Придет': 'V',
 'и': 'CONJ',
 'сразу': 'ADV',
 'прыг': 'ADV',
 'подоконник': 'S',
 'свесится': 'V',
 'из': 'PR',
 'окна': 'S',
 'смотрит': 'V',
 'Чай': 'S',
 'ему': 'SPRO',
 'тоже': 'PART',
 'туда': 'ADVPRO',
 'окно': 'S',
 'подавали': 'V',
 'Иной': 'APRO',
 'раз': 'S',
 'там': 'ADVPRO',
 'заночует': 'V',
 'Ему': 'SPRO',
 'даже': 'PART',
 'матрац': 'S',
 'купили': 'V',
 'специальный': 'A',
 'только': 'PART',
 'он': 'SPRO',
 'его': 'SPRO',
 'не': 'PART',
 'признавал': 'V',
 'К': 'PR',
 'чему': 'SPRO',
 'говорит': 'V',
 'такие': 'APRO',
 'роскоши': 'S',
 'И': 'CONJ',
 'спихнет': 'V',
 'с': 'PR',
 'подоконника': 'S',
 'А': 'CONJ',
 'потом': 'ADVPRO',
 'всю': 'APRO',
 'ночь': 'S',
 'вертится': 'V',
 'спать': 'V',
 'дает': 'V',
 'Гоголь

In [123]:
len(newdict_pos)

1355

In [124]:
len(morf.split())

2923